<div style="text-align: center;"> <span style="color:brown; font-weight:bold; font-size: 30px">Final Project- Sentiment Analysis using Amazon Reviews Dataset </span> </div>

Source: https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews

<span style="color:black; font-weight:bold; font-size: 18px">Setting up the environment, S3 and Athena Client</span>

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px">Importing Libraries</span>

In [1]:
# Importing Libraries

import boto3
import pandas as pd
import warnings

# Ignore only FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px">Setting up S3 Bucket and Key</span>

In [2]:
# AWS configuration
AWS_REGION = 'us-east-1'
S3_BUCKET_NAME = 'test-bucket-areena'

# Folder in S3 with my raw data
S3_KEY = 'final_project_data/'

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px">Setting up Athena</span>

In [3]:
# Athena Configuration
DATABASE_NAME = 'default'
s3_output_location = f's3://{S3_BUCKET_NAME}/query_results/'

# Initialize Athena client (no need to specify credentials)
athena_client = boto3.client('athena', region_name=AWS_REGION)

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px">Reading the Raw Dataset as a Pandas DataFrame</span>

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Training Dataset</span>

In [4]:
# Defining headers for the datasets
columns = ["polarity", "title", "description"]

# Path for Staged raw train data in s3
train_s3_path = f"s3://{S3_BUCKET_NAME}/{S3_KEY}train/train.csv"

# Reading staged csv data as pandas dataframe from raw train data
raw_train = pd.read_csv(train_s3_path, names=columns)

raw_train.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,polarity,title,description
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,Amazing!,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Testing Dataset</span>

In [5]:
# Path for Staged raw test data in s3
test_s3_path = f"s3://{S3_BUCKET_NAME}/{S3_KEY}test/test.csv"

# Reading staged data as pandas dataframe from raw test data
raw_test = pd.read_csv(test_s3_path, names=columns)

raw_test.head()

,polarity,title,description
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px">Data Types for Datasets</span>

In [6]:
# Checking data types to load into Athena
print(f"Training:\n{raw_train.dtypes}")
print(f"Testing:\n{raw_test.dtypes}")

Training:
polarity        int64
title          object
description    object
dtype: object
Testing:
polarity        int64
title          object
description    object
dtype: object


 <div style="text-align: center;"> <span style="color:blue; font-weight:bold; font-size: 24px"> Data Replication in S3 </span> </div>

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Training Dataset</span>

In [7]:
# Data Replication to S3 to ensure consistency, reliability and availability
s3_path = f's3://{S3_BUCKET_NAME}/test/train/raw-train.csv'
raw_train.to_csv(s3_path, index=False)

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Testing Dataset</span>

In [8]:
s3_path = f's3://{S3_BUCKET_NAME}/test/test/raw-test.csv'
raw_test.to_csv(s3_path, index=False)

Training Data Replicated at s3://test-bucket-areena/test/train

Testing Data Replicated at s3://test-bucket-areena/test/test

<div style="text-align: center;"> <span style="color:blue; font-weight:bold; font-size: 24px"> Querying in Athena </span> </div>

<span style="color:black; font-weight:bold; font-size: 18px">Creating Tables in Athena to load data from S3 </span>

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Training Datatable</span>

In [9]:
# Creating table to load train data into Athena
TABLE_NAME = 'train-datatable'

# table creation query
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS `{DATABASE_NAME}`.`{TABLE_NAME}` (
    polarity INT,
    title STRING,
    description STRING
)

ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
    "skip.header.line.count"="1"
)
LOCATION 's3://{S3_BUCKET_NAME}/test/train/'
""" 

In [10]:
# Execute query to create table
response = athena_client.start_query_execution(
    QueryString=create_table_query.strip(),
    QueryExecutionContext={'Database': DATABASE_NAME},
    ResultConfiguration={'OutputLocation': s3_output_location}
)

print("Athena Training Table creation query submitted.")

Athena Training Table creation query submitted.


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Testing Datatable</span>

In [11]:
# Creating table to load test data into Athena
TABLE_NAME = 'test-datatable'

# table creation query
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS `{DATABASE_NAME}`.`{TABLE_NAME}` (
    polarity INT,
    title STRING,
    description STRING
)

ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
    "skip.header.line.count"="1"
)
LOCATION 's3://{S3_BUCKET_NAME}/test/test/'
""" 

In [12]:
# Execute query to create table
response = athena_client.start_query_execution(
    QueryString=create_table_query.strip(),
    QueryExecutionContext={'Database': DATABASE_NAME},
    ResultConfiguration={'OutputLocation': s3_output_location}
)

print("Athena Training Table creation query submitted.")

Athena Training Table creation query submitted.


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px"> Query Function to avoid redundancy </span>

In [13]:
# Importing libraries
import time

# Function to run Athena query
def run_athena_query(query, database, output_location):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': output_location
        }
    )
    return response['QueryExecutionId']

# Function to check query status
def check_query_status(query_execution_id):
    while True:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        status = response['QueryExecution']['Status']['State']
        if status == 'SUCCEEDED':
            print("Query succeeded!")
            break
        elif status in ['FAILED', 'CANCELLED']:
            print(f"Query {status.lower()}.")
            raise Exception(f"Query failed or was cancelled: {response}")
        time.sleep(2)

# Function to get query results and convert to pandas DataFrame
def get_query_results_as_dataframe(query_execution_id):
    # Fetching the query results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iter = results_paginator.paginate(QueryExecutionId=query_execution_id)

    # Initialize a list to store rows and columns for DataFrame
    columns = []
    rows = []

    # Process the result pages
    for results_page in results_iter:
        # Get column info from the first page
        if not columns:
            columns = [col['Label'] for col in results_page['ResultSet']['ResultSetMetadata']['ColumnInfo']]

        # Skip the first row of the first page (column headers)
        for row in results_page['ResultSet']['Rows'][1:]:
            rows.append([col.get('VarCharValue', None) for col in row['Data']])

    # Create DataFrame from the results
    df = pd.DataFrame(rows, columns=columns)
    return df

In [14]:
# Execute queries using the methods
def main(query, DATABASE_NAME, s3_output_location):
    # Run the query
    query_execution_id = run_athena_query(query, DATABASE_NAME, s3_output_location)

    # Check the status of the query
    check_query_status(query_execution_id)

    # Fetch the results as a pandas DataFrame
    pd_df = get_query_results_as_dataframe(query_execution_id)

    return pd_df

<span style="color:black; font-weight:bold; font-size: 18px">Query to display Datatables</span>

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Training Datatable</span>

In [15]:
# Display original raw train datatable
query = f'SELECT * FROM "train-datatable";'

raw_train = main(query, DATABASE_NAME, s3_output_location)
raw_train.head()

Query succeeded!


,polarity,title,description
0,1,Lead in the Dryer's Power Cord,"The dryer looked good, but the power cord has ..."
1,2,WONDERFUL DRYER,I LOVE THIS HAIRDRYER! No more breaking dryer ...
2,1,Maybe I received a bad dryer but this did NOT ...,I did my research on a series of hair dryers p...
3,2,Excelente,Excelente producto funciona de maravilla mi ma...
4,1,The title's the best bit,"I've tried, believe me I've tried, but this is..."


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Testing Datatable</span>

In [17]:
# Display original raw test datatable
query = f'SELECT * FROM "test-datatable";'

raw_test = main(query, DATABASE_NAME, s3_output_location)

raw_test.head()

Query succeeded!


,polarity,title,description
0,1,Killing Time,"The plotline is interesting, but what happened..."
1,2,The thrill of a singer discovering herself,"An old girlfriend of mine, upon hearing this a..."
2,2,What a treat!,Balsam was a musician's musician. This CD is e...
3,2,Awesome gift!,"I got this for a friend of mine, and I wasn't ..."
4,1,"""DISAPPOINTED""","A (PS3)VIDEO GAME SHOULD BE DESIGNED AS/FOR ""F..."


<span style="color:black; font-weight:bold; font-size: 18px">Query to count rows in the Datatables</span>

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Training Data</span>

In [18]:
# Checking the row count in train data table
query = f'SELECT COUNT(*) FROM "train-datatable";'

count_train = main(query, DATABASE_NAME, s3_output_location)

count_train

Query succeeded!


,_col0
0,3600000


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Testing Data</span>

In [19]:
# Checking the row count in test data table
query = f'SELECT COUNT(*) FROM "test-datatable";'

count_test = main(query, DATABASE_NAME, s3_output_location)

count_test

Query succeeded!


,_col0
0,400000


<span style="color:black; font-weight:bold; font-size: 18px">Query to count duplicate rows from both datatables</span>

In [20]:
# Common rows in both the tables
query = f'SELECT * FROM "train-datatable" INTERSECT SELECT * FROM "test-datatable";'

intersect_train_test = main(query, DATABASE_NAME, s3_output_location)

intersect_train_test.head()

Query succeeded!


,polarity,title,description


No duplicates found 

<span style="color:black; font-weight:bold; font-size: 18px">Sampling the Datasets </span>

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Training Data</span>

In [24]:
# Sampling training dataset by 30%
query = f'SELECT * FROM "train-datatable" where RAND()<=0.3;'

train_sampled = main(query, DATABASE_NAME, s3_output_location)

train_sampled.head()

Query succeeded!


,polarity,title,description
0,1,Simply not a good movie.,I would have turned it off if I hadn't paid fo...
1,2,"If you love Oliver Stone, you will love this m...","Sit down, buckle up and hang on for a wild and..."
2,2,Savages don't make deals!,I liked this movie as I thought it was a good ...
3,1,Disappointing,"After his hugely enjoyable ""Natural Born Kille..."
4,1,Savages,Everyone(with the exception of Taylor Kitsch) ...


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 12px">Testing Data</span>

In [25]:
# Sampling testing dataset by 30%
query = f'SELECT * FROM "test-datatable" where RAND()<=0.3;'

test_sampled = main(query, DATABASE_NAME, s3_output_location)

test_sampled.head()

Query succeeded!


,polarity,title,description
0,1,Save your money buy something else,My company purchased 3 of these malfunctioning...
1,2,Damn this girl gots skills,This girl has got mad skills on the tables. Sh...
2,1,The record company won,I came on board with Too Far To Care (an unher...
3,2,Absolutely Outstanding,I purchased this CD after listening to a frien...
4,2,Great ideas if you want to make a sewing space,"I love the book ""Dream Sewing Spaces"" - which ..."


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px">Count rows in the Sampled DataFrames</span>

In [29]:
print("Rows in Sampled Train:", train_sampled.shape[0])
print("Rows in Sampled Testing:", test_sampled.shape[0])

Rows in Sampled Train: 1079489
Rows in Sampled Testing: 119326


<span style="color:black; font-weight:bold; font-size: 18px">Merging the Datasets</span>

In [30]:
combined_train_test = pd.concat([train_sampled, test_sampled], ignore_index=True)
print("Union of Training and Testing DataFrame reults in a Dataframe with:", combined_train_test.shape[0], "rows and ",combined_train_test.shape[1], "columns")

Union of Training and Testing DataFrame reults in a Dataframe with: 1198815 rows and  3 columns


<div style="text-align: center;"> <span style="color:blue; font-weight:bold; font-size: 24px"> Data Ingestion into S3 </span> </div>

In [31]:
# Staging the combined dataframe into S3 as a CSV for carrying out the Data Science Workflow Tasks

s3_combined_path = f's3://{S3_BUCKET_NAME}/test/combined/combined_train_test.csv'
combined_train_test.to_csv(s3_combined_path, index=False)

<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px">Creating Datatable for Combined Data</span>

In [41]:
TABLE_NAME = 'combined_train_test'

# table creation query
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS `{DATABASE_NAME}`.`{TABLE_NAME}` (
    polarity INT,
    title STRING,
    description STRING
)

ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
    "skip.header.line.count"="1"
)
LOCATION 's3://{S3_BUCKET_NAME}/test/combined/'
"""

# Execute query to create table
response = athena_client.start_query_execution(
    QueryString=create_table_query.strip(),
    QueryExecutionContext={'Database': DATABASE_NAME},
    ResultConfiguration={'OutputLocation': s3_output_location}
)

print("Athena Training Table creation query submitted.")

Athena Training Table creation query submitted.


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px">Display Combined Data</span>

In [42]:
query = f'SELECT * FROM "combined_train_test";'

combined_data = main(query, DATABASE_NAME, s3_output_location)

combined_data.head()

Query succeeded!


,polarity,title,description
0,2,Allright on the night/Buzzard,"A few months ago, I got a copy of the album, T..."
1,1,Beware watching this movie may cause impotency,This movie has to be the most boring movie tha...
2,1,A pessimistic view by an educated socialist,"21st Century Capitalsim, published in 1993, fa..."
3,2,EXCELLENT CD,I never really listened to the Mavericks but a...
4,2,One of Our Greatest Singers Delivers an Excell...,"Raul Malo is a great singer, and this is an ex..."


<span style="color:rgb(75,0,130); font-weight:bold; font-size: 16px">Display Number of Rows in Combined Data</span>

In [47]:
# Checking the row count in test data table
query = f'SELECT COUNT(*) AS "Count_Rows" FROM "combined_train_test";'

count_combined = main(query, DATABASE_NAME, s3_output_location)

count_combined

Query succeeded!


,Count_Rows
0,1198815


### Remaining Steps (EDA, Preprocessing, Transformation, Modeling and Evaluation) in the EMR Notebook  -->